In [120]:
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import datetime
import time
driver_path = 'C:/Users/taipj/chromedriver-win64'
driver = webdriver.Chrome()
data = []
driver.get("https://www.google.com/maps/@25.0036712,121.526284,15z?authuser=0&entry=ttu")

店名=[]
電話= []
地址=[]
今日營業時間=[]
經度=[]
緯度=[]




#查詢關鍵字
search_box = driver.find_element(By.NAME, 'q')  
search_box.clear()

search_box.send_keys("麵")

search_box.send_keys(Keys.RETURN)
time.sleep(5) 



#按下評分
element = driver.find_element(By.TAG_NAME, 'body')
driver.set_window_size(1200, 800)
actions = ActionChains(driver)
actions.move_to_element_with_offset(element, 0, 0)
actions.move_by_offset(-50, -280)  
actions.click_and_hold()
actions.release()
actions.perform()


time.sleep(2)  


#選擇星數
x=-230 #不限
x=-220 #2顆星 
x=-180 #2.5顆星 
x=-140 #3顆星
x=-100 #3.5顆星
x=-80 #4顆星
x=-40 #4.5顆星
element = driver.find_element(By.TAG_NAME, 'body')
driver.set_window_size(1200, 800)
actions = ActionChains(driver)
actions.move_to_element_with_offset(element, 0, 0)
actions.move_by_offset(-50, x) 
actions.click_and_hold()  
actions.release()
actions.perform()


time.sleep(1)  

           
#選擇營業中
element = driver.find_element(By.TAG_NAME, 'body')
driver.set_window_size(1200, 800)
actions = ActionChains(driver)
actions.move_to_element_with_offset(element, 0, 0)
actions.move_by_offset(50, -280)  
actions.click_and_hold()  
actions.release()
actions.perform()


time.sleep(1)  

elementm = driver.find_element(By.TAG_NAME, 'body')
driver.set_window_size(1200, 800)
actions = ActionChains(driver)
actions.move_to_element_with_offset(elementm, 0, 0)
actions.move_by_offset(100, -220) 
actions.click_and_hold()  
actions.release()
actions.perform()
actions.move_to_element_with_offset(elementm, 0, 0)
actions.move_by_offset(330, 220) 
actions.click_and_hold()  
actions.release()
actions.perform()

time.sleep(1)  

driver.set_window_size(1200, 800)
actions = ActionChains(driver)
actions.move_to_element_with_offset(elementm, 0, 0)
actions.move_by_offset(-114, 280) 
actions.click_and_hold()  
actions.perform()
time.sleep(5)
actions.release()
actions.perform()




           
elementf = driver.find_elements(By.CLASS_NAME, 'hfpxzc')
i = 1
for i in range(5):

    second_element = elementf[i]
    driver.execute_script("arguments[0].click();", second_element)
    time.sleep(0.5)
        
        
    html = driver.page_source
    time.sleep(0.5)
    
    #找店名
    pattern = r'<title>(.*?)<\/title>'
    match = re.search(pattern, html)
    if match:
        title_text = match.group(1)
        cleaned_title = re.sub(r'\s*-\s*Google\s*地圖\s*', '', title_text)
        店名.append(cleaned_title)
    
    
    #找地址
    address_element = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.rogA2c .Io6YTe'))
    )
    
    # 获取地址元素的文字内容
    address = address_element.text.strip()
    地址.append(address)

    
    #找電話
    phone_number_start = html.find('AeaXub')
    phone_number_end = html.find('</div>', phone_number_start)
    if phone_number_start != -1 and phone_number_end != -1:
        phone_number_element = html[phone_number_start:phone_number_end]
        phone_number = phone_number_element.split('>')[1].strip()




    phone_number_match = re.search(r'(\d{2}\s\d{4}\s\d{4})', html)
    if phone_number_match:
        phone_number = phone_number_match.group(0)
        電話.append(phone_number)
       
    else:
        電話.append("未提供電話")



   

    #找經緯度
    current_url = driver.current_url
    # 使用正則表達式提取經緯度
    pattern = r'@(-?\d+\.\d+),(-?\d+\.\d+),'
    match = re.search(pattern, current_url)

    if match:
        latitude = match.group(1)
        longitude = match.group(2)
        緯度.append(latitude)
        經度.append(longitude) 


    
    
    
    #找營業時間
    today = datetime.datetime.today()
    weekday = today.weekday()
    if weekday == 0 :
        week = "一"
    if weekday == 1 :
        week = "二"
    if weekday == 2 :
        week = "三"
    if weekday == 3 :
        week = "四"
    if weekday == 4 :
        week = "五"
    if weekday == 5 :
        week = "六"
    if weekday == 6 :
        week = "日"
    try:
        open_hours_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '[aria-label="營業時間"]'))
        )
        open_hours_button.click()
        time.sleep(1)
    except:
        print("Opening hours button not found or clickable.")
    pattern = r"星期" + week + "、(.*?)[;.]"
    
    match = re.search(pattern, html)
    if match:
        今日營業時間.append(match.group(1)) 

    else:
        今日營業時間.append("未找到匹配內容")
        
        
        
    try:
        more_comments_button = driver.find_element(By.XPATH, "//*[contains(text(), '更多評論')]")
        more_comments_button.click()
    except:
        print("點擊失敗")
    time.sleep(2)
    driver.set_window_size(1200, 800)
    actions = ActionChains(driver)
    actions.move_to_element_with_offset(elementm, 0, 0)
    actions.move_by_offset(265, 300) 
    actions.click_and_hold()
    actions.perform()
    time.sleep(3)
    actions.release()
    actions.perform()
    def scrape_data():
        名字 = []
        星數 = []
        評論 = []

        # 查找class為jftiEf fontBodyMedium的所有元素來獲取名字
        name_elements = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
        for element in name_elements:
            aria_label = element.get_attribute('aria-label')
            名字.append(aria_label)

        # 查找class為kvMYJc的所有元素來獲取星數
        star_elements = driver.find_elements(By.CLASS_NAME, 'kvMYJc')
        for element in star_elements:
            aria_label = element.get_attribute('aria-label')
            星數.append(aria_label)

        # 查找class為jftiEf fontBodyMedium的所有元素來獲取評論
        comment_elements = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
        for element in comment_elements:
            sub_elements = element.find_elements(By.CSS_SELECTOR, 'span.wiI7pd')
            if sub_elements:
                for sub_element in sub_elements:
                    content = sub_element.text
                    評論.append(content)
            else:
                評論.append("無評論")

        return 名字, 星數, 評論

    # 主循環，直到三者數量相同
    while True:
        名字, 星數, 評論 = scrape_data()
        if len(名字) == len(星數) == len(評論):
            break
        else:
            print("數據不一致，重新抓取")
           

    # 將抓取的數據轉換為DataFrame
    店家評論 = pd.DataFrame({"名字": 名字, "星數": 星數, "評論": 評論})

    # 列印抓取的評論
    print(店家評論)





df = pd.DataFrame({"店名" : 店名 , "電話" : 電話 , "地址" : 地址 , "今日營業時間" : 今日營業時間 , "經度" : 經度 , "緯度" : 緯度 })
print(df)



                 名字    星數                                                 評論
0       Carlos Chen  4 顆星               其實蠻好吃的！\n但就是太小份 也稍貴\n但宵夜時段算是還可以到選擇 …
1              CN L  5 顆星  點了兩碗小碗大腸麵線、一份肉圓、一份豬血糕、一份小的甜不辣，每一碗都是 55 元～\n我覺得...
2               林安琪  3 顆星  麵線算是普通，肉圓算是好吃，蘿蔔湯貢丸是沒熟還是沒煮透，不好吃，沒吃完，白蘿蔔也很硬。我個人...
3               賴淑玲  1 顆星            肉圓55…清麵線大碗60…價錢變貴很多（大碗小碗都一樣小），但吃起來卻變難吃。
4     Yonton Kormar  2 顆星  怎麼味道都變了，難吃！\n面線的腸子很少，腸子很少，醋的味道很不好！\n肉圓的味道也不如以往...
5     noodles Super  4 顆星  位子較小\n\n本來對白大腸沒啥興趣 但還有味道 麵線較有甜味 寸土寸金的店面 是經營24小...
6         HSU Julia  3 顆星  2023.07.01\n連鎖店，google評論送優惠券（我沒有用評論拿優惠券），所以評論真...
7          Donny Yu  5 顆星  大腸麵線類似阿宗麵線，湯頭還不錯，大腸清洗乾淨不是用滷的所以偏白帶著一點Q勁。\n環境上整潔...
8      Dennis Huang  5 顆星                        冬天24小時有宵夜超感謝！！\n第二次吃了，口味穩定~
9               黎志強  4 顆星       自從十多年前、被取締舉發拆除以及各種莫名原因的臨檢叨擾後，就很少再來師大商圈逛逛了。 …
10         debbie w  4 顆星  沒有內用坐椅，但騎樓下有桌子，椅子可以吃東西，麵線好吃，甜不辣也不錯，唯一有甜不辣煮太久，感...
11       李浩宇（Cross）  5 顆星                  份量小巧精緻，味道清淡樸實。\n\n重點是24小時營業，夜貓友善。

               名字    星數                                                 評論
0        Jing Yeh  5 顆星  在台灣算是非常日式的拉麵店，據說是日本人開的拉麵店但是不是日本連鎖的。推薦：沾麵>雞白湯>柚...
1             LZG  4 顆星  店內座位數蠻多的，宵夜時段（大概晚上22點）等餐時間大約是10分鐘左右，湯頭挺不錯的但麵條口...
2             許睿宸  5 顆星  今日晚餐，雞白湯拉麵，柚香雞白湯拉麵，叉燒飯，炸蓮藕，煎餃子。\n以上消費符合店家規定的每人...
3    shihying Lin  5 顆星  第一次拜訪光華店的千雲！以往都是去林森店吃！排隊都要排非常久！今天很幸運的不用等就立刻可以入...
4            Cody  5 顆星  麵屋千雲是一家位於市中心的拉麵店，以其濃郁的湯頭和彈牙的麵條而聞名。進入餐廳，首先映入眼簾的...
5             샤오후  5 顆星                              推薦 ⭐️⭐️⭐️⭐️⭐️\n\n👉🏻 …
6           kenny  5 顆星  最近這一兩年，不論去日本或是在台灣，也吃了不下10家以上的拉麵吧！\n\n這應該是這段期間吃...
7            Alin  4 顆星  今天點了888的雙人套餐外加點ㄧ碗白飯，有兩碗拉麵，雞白湯和豚骨湯兩種口味，另有ㄧ份唐揚雞、...
8          SC Liu  5 顆星  店面明顯好找，下午四點多小朋友下課來不用排隊很多座位，服務人員很快帶我們入座，點餐使用掃碼點...
9    Chenwei Yang  5 顆星  空間很大很乾淨，拉麵的湯頭濃厚但相當順口，叉燒柔軟沒有腥味，很好吃的一碗麵\n餃子香氣和口感...
10     Wenli Liao  5 顆星  塩味系，喜歡濃郁鹹香的人可以品嚐，風味都蠻有特色&實在感，服務上很特別還有兒童圍兜、髮圈可以...
11     Cynnie Liu  4 顆星      店面很乾淨舒適，麵條好吃，餃子內餡跟湯都超鹹，可以跟服務人員索取原味高湯稀釋再喝會比較順口
12      章魚（うさぎさん）  5 顆星  